In [29]:
# Tool Calling Workflow (TCW) - A tool for managing and executing workflows
#1) Tool Definition
#2) Tool Binding
#3) Tool Execution
#4) The llm uses the tool to execute the workflow

In [30]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
llm.invoke("What is the weather in Trivandrum?")

AIMessage(content="I don't have real-time data access to provide current weather information. To find the latest weather updates for Trivandrum, I recommend checking a reliable weather website or using a weather app.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 17, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BwAZr3LVfqS6GWx6laZnkSNUcC9OC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f04eca8a-effa-4910-a9f9-c04d255f42d0-0', usage_metadata={'input_tokens': 17, 'output_tokens': 38, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [32]:
from langchain_core.tools import tool

In [33]:
@tool
def get_weather(location: str) -> str:
    """
    A mock weather API that returns the weather for a given location.
    """
    return f"The weather in {location} is sunny with a high of 30°C."

In [34]:
get_weather.invoke({"location": "Trivandrum"})

'The weather in Trivandrum is sunny with a high of 30°C.'

In [35]:
get_weather

StructuredTool(name='get_weather', description='A mock weather API that returns the weather for a given location.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x000002B653920CC0>)

In [36]:
tools = [get_weather]

In [37]:
llm_with_tools = llm.bind_tools(tools)

In [38]:
result = llm_with_tools.invoke("What is the weather in Trivandrum?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MtLSnQyOC8JBkmv3O9m5TOsR', 'function': {'arguments': '{"location":"Trivandrum"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BwAZsBrzcQwMbPhukED5wHLLYeQpc', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bebccd31-7708-49dd-8ebf-b5b4c6f2d7d8-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Trivandrum'}, 'id': 'call_MtLSnQyOC8JBkmv3O9m5TOsR', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {

In [39]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Trivandrum'},
  'id': 'call_MtLSnQyOC8JBkmv3O9m5TOsR',
  'type': 'tool_call'}]

In [40]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [HumanMessage(content="What is the weather in Trivandrum?")]

llm_output = llm_with_tools.invoke(messages)
llm_output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ewhZ7xNCuBWgosaNncYXHUNb', 'function': {'arguments': '{"location":"Trivandrum"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 60, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BwAZtx20uY5RNhheZF3K0MMsi5X9z', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9affcd2b-2d39-4d2b-b99c-49eb579f3d19-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Trivandrum'}, 'id': 'call_ewhZ7xNCuBWgosaNncYXHUNb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 17, 'total_tokens': 77, 'input_token_details': {

In [41]:
tool_mapping = {"get_weather": get_weather}

In [42]:
for tool_call in result.tool_calls:
    tool = tool_mapping[tool_call['name'].lower()]
    tool_output = tool.invoke(tool_call['args'])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call['id']))

In [43]:
messages

[HumanMessage(content='What is the weather in Trivandrum?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='The weather in Trivandrum is sunny with a high of 30°C.', tool_call_id='call_MtLSnQyOC8JBkmv3O9m5TOsR')]

In [44]:
tool_call

{'name': 'get_weather',
 'args': {'location': 'Trivandrum'},
 'id': 'call_MtLSnQyOC8JBkmv3O9m5TOsR',
 'type': 'tool_call'}

In [45]:
tool_call['name'].lower()

'get_weather'

In [46]:
tool_mapping[tool_call['name'].lower()]

StructuredTool(name='get_weather', description='A mock weather API that returns the weather for a given location.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x000002B653920CC0>)

In [47]:
get_weather

StructuredTool(name='get_weather', description='A mock weather API that returns the weather for a given location.', args_schema=<class 'langchain_core.utils.pydantic.get_weather'>, func=<function get_weather at 0x000002B653920CC0>)

In [48]:
messages

[HumanMessage(content='What is the weather in Trivandrum?', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='The weather in Trivandrum is sunny with a high of 30°C.', tool_call_id='call_MtLSnQyOC8JBkmv3O9m5TOsR')]

In [51]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, initialize_agent

In [52]:
agent = initialize_agent(llm=llm, tools=tools)

C:\Users\ajite\AppData\Local\Temp\ipykernel_21080\2670483501.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(llm=llm, tools=tools)


In [53]:
agent.invoke("What is the weather in Trivandrum?")

{'input': 'What is the weather in Trivandrum?',
 'output': 'The weather in Trivandrum is sunny with a high of 30°C.'}